<a href="https://colab.research.google.com/github/kyokicchi/ggl_an_codes/blob/master/LGBmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Train model

In [1]:
!pip install lightgbm

    100% |████████████████████████████████| 1.1MB 7.6MB/s 


In [0]:
import lightgbm as lgb
import numpy as np
import pandas as pd
from google.colab import drive
from google.colab import files
from sklearn.model_selection import train_test_split
import pickle

In [3]:
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
%cd ../content/gdrive/My Drive/projects/ggl_an/
%ls

/content/gdrive/My Drive/projects/ggl_an
model_LGB.sav  output_LGB.csv  user_test.csv    user_train.csv
model_NN.h5    output_NN.csv   user_test_n.csv  user_train_n.csv


In [0]:
s_data = 'user_train.csv'
filename = 'model_LGB.sav'
s_col_tgt = 'transactionRevenue'
l_col_drop = [s_col_tgt]

In [6]:
%%time
df_train = pd.read_csv(s_data, index_col = 0)

<string>:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: user 2.53 s, sys: 348 ms, total: 2.87 s
Wall time: 3.65 s


In [7]:
%%time
df_y = df_train[s_col_tgt]
df_x = df_train.drop(l_col_drop, axis=1)

CPU times: user 33.9 ms, sys: 57 ms, total: 90.9 ms
Wall time: 508 ms


In [8]:
%%time
x_train, x_test, y_train, y_test = train_test_split(df_x, df_y)



CPU times: user 594 ms, sys: 63.2 ms, total: 658 ms
Wall time: 656 ms


In [9]:
%%time
from sklearn.model_selection import GridSearchCV

def applyGSCV(model, param, X, Y):
    res = GridSearchCV(model, param, cv=3)
    res.fit(X, Y)
    return res


model = lgb.LGBMRegressor()

param = {
    'learning_rate': [0.1,0.15],
    'n_estimators': [200],
    'num_leaves': [100],
}
  
model_LGB = applyGSCV(model, param, x_train, y_train)



CPU times: user 3min 48s, sys: 2.16 s, total: 3min 50s
Wall time: 1min 58s


In [10]:
print(model_LGB.best_score_)
print(model_LGB.best_estimator_)


0.3918641867971566
LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       importance_type='split', learning_rate=0.1, max_depth=-1,
       min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
       n_estimators=200, n_jobs=-1, num_leaves=100, objective=None,
       random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
       subsample=1.0, subsample_for_bin=200000, subsample_freq=0)


In [11]:
%%time
model = lgb.LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       importance_type='split', learning_rate=0.1, max_depth=-1,
       min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
       n_estimators=200, n_jobs=-1, num_leaves=100, objective=None,
       random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
       subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

model.fit(x_train, 
          y_train,
          eval_set=[(x_train, y_train),(x_test, y_test)],
          eval_metric='rmse',
          early_stopping_rounds = 300)

[1]	training's l2: 7.26132	training's rmse: 2.69468	valid_1's l2: 9.08666	valid_1's rmse: 3.01441
Training until validation scores don't improve for 300 rounds.
[2]	training's l2: 6.72163	training's rmse: 2.59261	valid_1's l2: 8.54035	valid_1's rmse: 2.92239
[3]	training's l2: 6.27734	training's rmse: 2.50546	valid_1's l2: 8.08677	valid_1's rmse: 2.84372
[4]	training's l2: 5.91366	training's rmse: 2.4318	valid_1's l2: 7.71574	valid_1's rmse: 2.77772
[5]	training's l2: 5.60848	training's rmse: 2.36822	valid_1's l2: 7.40719	valid_1's rmse: 2.72161
[6]	training's l2: 5.35547	training's rmse: 2.31419	valid_1's l2: 7.15014	valid_1's rmse: 2.67397
[7]	training's l2: 5.14079	training's rmse: 2.26733	valid_1's l2: 6.93721	valid_1's rmse: 2.63386
[8]	training's l2: 4.96031	training's rmse: 2.22718	valid_1's l2: 6.74867	valid_1's rmse: 2.59782
[9]	training's l2: 4.80322	training's rmse: 2.19163	valid_1's l2: 6.60435	valid_1's rmse: 2.56989
[10]	training's l2: 4.67254	training's rmse: 2.16161	val

In [12]:
%%time
pickle.dump(model_LGB, open(filename, 'wb'))


CPU times: user 184 ms, sys: 4.02 ms, total: 188 ms
Wall time: 366 ms


# Predict / Output

In [13]:
%%time

model = pickle.load(open('model_LGB.sav', 'rb'))

CPU times: user 32.5 ms, sys: 2.96 ms, total: 35.4 ms
Wall time: 29.6 ms


In [14]:
%%time
df_user_train = pd.read_csv('user_train.csv', index_col = 0, dtype={'fullVisitorId': 'str'})
df_user_test = pd.read_csv('user_test.csv', index_col = 0, dtype={'fullVisitorId': 'str'})


CPU times: user 4.76 s, sys: 247 ms, total: 5.01 s
Wall time: 5.79 s


In [15]:

df_y = df_user_train['transactionRevenue']
df_x = df_user_train.drop(['transactionRevenue'], axis=1)
df_tgt = df_user_test.drop(['transactionRevenue'], axis=1)

df_x.shape, df_y.shape, df_tgt.shape


((714167, 23), (714167,), (617242, 23))

In [17]:
%%time

from sklearn.metrics import mean_squared_error

pred = model.predict(df_x)
mse = mean_squared_error(df_y, pred)
rmse = np.sqrt(mse)
print(rmse)

1.825505501479145
CPU times: user 22.3 s, sys: 67.8 ms, total: 22.4 s
Wall time: 11.5 s


In [18]:
%%time
pred_tgt = model.predict(df_tgt)


CPU times: user 19.2 s, sys: 63.9 ms, total: 19.2 s
Wall time: 9.86 s


In [20]:
pred_tgt.min(),pred_tgt.max(),len(pred_tgt[pred_tgt>0]) / len(pred_tgt)  

(-17.62574618750795, 123.55485937634258, 0.7848671995748832)

In [0]:
pred_tgt[pred_tgt<0] = 0
df_out = pd.DataFrame(index = df_tgt.index)
df_out['PredictedLogRevenue'] = pred_tgt


In [22]:
df_out.head()

,PredictedLogRevenue
fullVisitorId,
0000000259678714014,0.501595
0000049363351866189,0.000000
0000053049821714864,0.000000
0000059488412965267,0.000545
0000085840370633780,0.000239


In [23]:
df_out.describe()

,PredictedLogRevenue
count,617242.000000
mean,0.195767
std,1.377942
min,0.000000
25%,0.000239
50%,0.000239
75%,0.001224
max,123.554859


In [24]:
%%time
df_out.to_csv('output_LGB.csv')

CPU times: user 1.34 s, sys: 29 ms, total: 1.37 s
Wall time: 2.08 s
